# Instantiating Convulutional Network (convnet)

In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]

In [10]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(224, 224, 3)) # width, height, channels
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu") (inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten() (x)
outputs = layers.Dense(8, activation="softmax")(x) # 8 classes in AffectNet
model = keras.Model(inputs=inputs, outputs=outputs)

In [9]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 52, 52, 128)       7385

# Load AffectNet Dataset

In [4]:
# GDrive einbinden (falls on Colab)
#from google.colab import drive
#drive.mount('/content/drive')

In [5]:
import os
import numpy as np
from natsort import natsorted
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import array_to_img

# local
train_img_dir = r'C:\Users\LEAND\Coding\_FER\AffectNet\train_set\images'
train_label_dir = r'C:\Users\LEAND\Coding\_FER\AffectNet\train_set\annotations'
test_img_dir = r'C:\Users\LEAND\Coding\_FER\AffectNet\val_set\images'
test_label_dir = r'C:\Users\LEAND\Coding\_FER\AffectNet\val_set\annotations'

# GDrive
#train_img_dir = '/content/drive/Othercomputers/MeinComputer/_FER/AffectNet/train_set/images/'
#train_label_dir = '/content/drive/Othercomputers/MeinComputer/_FER/AffectNet/train_set/annotations/'
#test_img_dir = '/content/drive/Othercomputers/MeinComputer/_FER/AffectNet/val_set/images/'
#test_label_dir = '/content/drive/Othercomputers/MeinComputer/_FER/AffectNet/val_set/annotations/'

def load_data(train_img_dir, train_label_dir, test_img_dir, test_label_dir, limit_train=None, limit_test=None):
    x_train, y_train, x_test, y_test = [], [], [], []

    x_train_files = os.listdir(train_img_dir)
    x_train_files = natsorted(x_train_files)

    y_train_files = os.listdir(train_label_dir)
    y_train_files = natsorted(y_train_files)

    x_test_files = os.listdir(test_img_dir)
    x_test_files = natsorted(x_test_files)

    y_test_files = os.listdir(test_label_dir)
    y_test_files = natsorted(y_test_files)

    for i, file in enumerate(x_train_files):
        if i == limit_train:
            break
        file_path = os.path.join(train_img_dir, file)
        img = load_img(file_path)
        data = img_to_array(img)
        x_train.append(data)

    for i, file in enumerate(y_train_files):
        if i == limit_train * 4: # four files per image
            break
        if "_exp.npy" in file:
            file_path = os.path.join(train_label_dir, file)
            data = np.load(file_path)
            y_train.append(data)

    for i, file in enumerate(x_test_files):
        if i == limit_test:
            break
        file_path = os.path.join(test_img_dir, file)
        img = load_img(file_path)
        data = img_to_array(img)
        x_test.append(data)

    for i, file in enumerate(y_test_files):
        if i == limit_test:
            break
        if "_exp.npy" in file:
            file_path = os.path.join(test_label_dir, file)
            data = np.load(file_path)
            y_test.append(data)

    x_train, y_train, x_test, y_test = np.array(x_train), np.array(y_train).astype(int), np.array(x_test), np.array(y_test).astype(int)

    return (x_train, y_train), (x_test, y_test)

In [6]:
(train_images, train_labels), (test_images, test_labels) = load_data(train_img_dir,
                                                                     train_label_dir,
                                                                     test_img_dir,
                                                                     test_label_dir,
                                                                     limit_train=10000,
                                                                     limit_test=None)

In [11]:
# TODO:
# learning-rate runter (optimizer)
# batch-size erhöhen
# validation set statt testset
# model größer bauen bis overfitting, dann wieder langsam zurückbauen

train_images = train_images.reshape((10000, 224, 224, 3))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((3999, 224, 224, 3))
test_images = test_images.astype("float32") / 255

model.compile(optimizer="rmsprop",
              loss = "sparse_categorical_crossentropy",
              metrics = ["accuracy"])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
157/157 [==============================] - 178s 1s/step - loss: 1.5240 - accuracy: 0.4492
Epoch 2/5
157/157 [==============================] - 181s 1s/step - loss: 1.4963 - accuracy: 0.4573
Epoch 3/5
157/157 [==============================] - 176s 1s/step - loss: 1.4950 - accuracy: 0.4609
Epoch 4/5
157/157 [==============================] - 175s 1s/step - loss: 1.4923 - accuracy: 0.4609
Epoch 5/5
157/157 [==============================] - 173s 1s/step - loss: 1.4905 - accuracy: 0.4609


In [12]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

125/125 [==============================] - 18s 144ms/step - loss: 3.0008 - accuracy: 0.1250
Test accuracy: 0.125
